In [5]:
# 定点数
# 线性映射
    
    ### 对称量化
    ### 非对称量化

# 非线性映射 DSQ
# 非常抱歉，我理解有误。以下是一些关于非线性映射量化的学习资源：

# 网站：
# arXiv：https://arxiv.org/
# IEEE Xplore：https://ieeexplore.ieee.org/
# ACM Digital Library：https://dl.acm.org/
# ScienceDirect：https://www.sciencedirect.com/
# ResearchGate：https://www.researchgate.net/
# Google Scholar：https://scholar.google.com/
# GitHub：https://github.com/
# 推荐论文：
# "Non-Linear Quantization for Deep Neural Networks" by Suyog Gupta, Ankur Agrawal, Kailash Gopalakrishnan, Pritish Narayanan (arXiv)：https://arxiv.org/abs/1609.07061
# "Learning Nonlinear Quantization Maps for Near-Optimal Decision Making" by Peter F. Christoffersen and David A. Castañón (IEEE Transactions on Neural Networks and Learning Systems)：https://ieeexplore.ieee.org/document/7445541
# "Nonlinear Compressed Sensing via Deep Neural Networks" by Yiran Zhong and Shuchin Aeron (IEEE Transactions on Signal Processing)：https://ieeexplore.ieee.org/document/8808179
# "Nonlinear Mapping for Data Visualization: A Review" by Elham Khabiri and Yvan J. Bédard (Journal of Visualization)：https://link.springer.com/article/10.1007/s12650-019-00597-x
# "Nonlinear Dimensionality Reduction and Visualization of High-Dimensional Data via Self-Organizing Maps" by Teuvo Kohonen (Proceedings of the International Neural Network Conference)：https://link.springer.com/chapter/10.1007/3-540-44668-0_33
# 希望这些资源能够帮助您学习非线性映射量化。

In [2]:
max_value = 127
x = 0.612
remainder = x / max_value
int8_x = x / remainder
print(f"int8_x: {int8_x}")

int8_x: 127.0


In [4]:
# int8_x --> float

f32_x = int8_x * remainder
print(f"f32_x: {f32_x}")

f32_x: 0.612


In [21]:
# --- 对称量化

In [7]:
import numpy as np

In [67]:
np.random.seed(3)

x = np.random.randn(3)
print(x)

[1.78862847 0.43650985 0.09649747]


In [68]:
# np.round(x[1])  四舍五入

In [70]:
# ---- 基础的对称量化法
def get_scale(x):
    scale = (np.max(x) - np.min(x)) / (127-(-127))
    return scale

def quant_x(x, scale):
    q = np.round(x / scale)
    return q

def dequant_x(x, scale):
    r = x * scale
    return r

def saturation_x(x):
    x = np.clip(x, a_max=127, a_min=-128)
    return x
    
print(f"x: {x} ")    
scale = get_scale(x)
print(f"scale:{scale}")
q = quant_x(x, scale)
q = saturation_x(q)

print(f"Q:{q}")

R = dequant_x(q, scale)
print(f"R:{R}")

x: [1.78862847 0.43650985 0.09649747] 
scale:0.0066619330919618506
Q:[127.  66.  14.]
R:[0.8460655  0.43968758 0.09326706]


In [71]:
# 由此可见, 采用简单的对称量化，不进行偏移和最大值量化, 浮点数量化为 in8整数，可能会发生阶段;
# 针对阶段带来的误差问题，我们可以采用 偏移和最大值量化解决；


In [72]:
# ---- 偏移法

# np.round(x[1])  四舍五入

def get_scale(x):
    scale = (np.max(x) - np.min(x)) / (127-(-127))
    return scale

def quant_x(x, scale):
    q = np.round(x / scale)
    return q

def compute_z(q, scale, r):
    q = np.abs(q)
    r = np.abs(r)
    z = np.max(q) - saturation_x(np.max(r)/scale)
    return -z

def dequant_x(x, scale, z):
    r = (x + z)* scale
    return r

def saturation_x(x):
    x = np.clip(x, a_max=127, a_min=-128)
    return x

def get_new_q(x, scale, z):
    return saturation_x(x / scale - z)
    
print(f"x: {x}")    
scale = get_scale(x)
print(f"scale:{scale}")
q = quant_x(x, scale)
print(f"Q:{q}")

# z 是
z = compute_z(q, scale, x)
print(f"z: {z}")

q = get_new_q(x, scale, z)
print(f"new_q: {q}")

R = dequant_x(q, scale, z)
print(f"R:{R}")

# 实现的有个小bug, 正负值情况没有考虑清楚

x: [1.78862847 0.43650985 0.09649747]
scale:0.0066619330919618506
Q:[268.  66.  14.]
z: -141.0
new_q: [127. 127. 127.]
R:[-0.09326706 -0.09326706 -0.09326706]


In [74]:
# ---- 最大绝对值法

# np.round(x[1])  四舍五入

def get_scale(x):
    scale = (np.max(x)) / (2*127)
    return scale

def quant_x(x, scale):
    q = np.round(x / scale)
    return q

def compute_z(q, scale, r):
    q = np.abs(q)
    r = np.abs(r)
    z = np.max(q) - saturation_x(np.max(r)/scale)
    return -z

def dequant_x_offset(x, scale, z):
    r = (x + z)* scale
    return r

def dequant_x(x, scale):
    r = (x) * scale
    return r

def saturation_x(x):
    x = np.clip(x, a_max=127, a_min=-128)
    return x

def get_new_q(x, scale, z):
    return saturation_x(x / scale - z)
    
print(f"x: {x}")    
scale = get_scale(x)
print(f"scale:{scale}")
q = quant_x(x, scale)
print(f"Q:{q}")

# z 是
z = compute_z(q, scale, x)
print(f"z: {z}")

# q = get_new_q(x, scale, z)
# print(f"new_q: {q}")

R = dequant_x(q, scale)
print(f"R:{R}")

# 实现的有个小bug, 正负值情况没有考虑清楚

x: [1.78862847 0.43650985 0.09649747]
scale:0.007041844383583931
Q:[254.  62.  14.]
z: -127.0
R:[1.78862847 0.43659435 0.09858582]
